In [13]:
import pandas as pd
import json

# --- CONFIG ---
input_file = "nodes_and_links.ods"
output_dir = "."   # can be changed if you want to save elsewhere

# --- READ SHEETS ---
nodes_df = pd.read_excel(input_file, sheet_name="Nodes", engine="odf")
links_df = pd.read_excel(input_file, sheet_name="Links", engine="odf")
cbbs_df  = pd.read_excel(input_file, sheet_name="CBB_list", engine="odf")
objectives_df = pd.read_excel(input_file, sheet_name="Objectives", engine="odf")
artifacts_df = pd.read_excel(input_file, sheet_name="Artifacts", engine="odf")

# --- CLEAN BASIC STRUCTURE ---
nodes_df = nodes_df.fillna("")
links_df = links_df.fillna("")
cbbs_df  = cbbs_df.fillna("")
objectives_df = objectives_df.fillna("")
artifacts_df = artifacts_df.fillna("")

# --- CONVERT TO DICTIONARIES ---
nodes_json = nodes_df.to_dict(orient="records")
links_json = links_df.to_dict(orient="records")

# --- CBB STRUCTURE ---
# Combine cbbs and definitions per id
cbbs_grouped = (
    cbbs_df.groupby("id")
    .apply(lambda x: [
        {"cbb": c, "definition": d} 
        for c, d in zip(x["cbbs"], x["definitions"])
    ])
    .to_dict()
)

cbbs_json = [{"id": node_id, "cbbs": items} for node_id, items in cbbs_grouped.items()]


def process_objectives(df):
    """Convert a sheet with id + multiline text columns into a JSON-friendly format."""
    df = df.dropna(subset=["id"])  # Remove blank rows
    df["id"] = df["id"].astype(int)

    records = []
    for _, row in df.iterrows():
        # Split text on newlines or semicolons
        items = []
        if pd.notna(row.iloc[1]):  # Assuming the second column has the data
            text = str(row.iloc[1]).strip()
            # Split on newline or semicolon
            for item in [x.strip() for x in text.replace("\r", "").split("\n") if x.strip()]:
                items.append(item)
        records.append({"id": row["id"], "objectives": items})

    return records

def process_artifacts(df):
    """Convert a sheet with id + multiline text columns into a JSON-friendly format."""
    df = df.dropna(subset=["id"])  # Remove blank rows
    df["id"] = df["id"].astype(int)

    records = []
    for _, row in df.iterrows():
        # Split text on newlines or semicolons
        items = []
        if pd.notna(row.iloc[1]):  # Assuming the second column has the data
            text = str(row.iloc[1]).strip()
            # Split on newline or semicolon
            for item in [x.strip() for x in text.replace("\r", "").split("\n") if x.strip()]:
                items.append(item)
        records.append({"id": row["id"], "artifacts": items})

    return records

# Process the two sheets
objectives_json = process_objectives(objectives_df)
artifacts_json = process_artifacts(artifacts_df)

# --- SAVE TO JSON FILES ---
with open(f"{output_dir}/nodes.json", "w", encoding="utf-8") as f:
    json.dump(nodes_json, f, indent=2, ensure_ascii=False)

with open(f"{output_dir}/links.json", "w", encoding="utf-8") as f:
    json.dump(links_json, f, indent=2, ensure_ascii=False)

with open(f"{output_dir}/cbblinks.json", "w", encoding="utf-8") as f:
    json.dump(cbbs_json, f, indent=2, ensure_ascii=False)
    
with open(f"{output_dir}/objectives.json", "w", encoding="utf-8") as f:
    json.dump(objectives_json, f, indent=2, ensure_ascii=False)
    
with open(f"{output_dir}/artifacts.json", "w", encoding="utf-8") as f:
    json.dump(artifacts_json, f, indent=2, ensure_ascii=False)

print("✅ JSON files created: nodes.json, links.json, cbblinks.json, objectives.json, artifacts.json")


✅ JSON files created: nodes.json, links.json, cbblinks.json, objectives.json, artifacts.json


In [14]:
from collections import defaultdict

# --- Load your existing objectives.json ---
with open("objectives.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Group by id ---
grouped = defaultdict(list)
for entry in data:
    grouped[str(entry["id"])].extend(entry["objectives"])

# --- Convert to the correct format ---
result = [{"id": k, "objectives": v} for k, v in grouped.items()]

# --- Save back to a new JSON file ---
with open("objectives_grouped.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print("✅ Grouped objectives.json written successfully.")
print(f"📦 Total unique IDs: {len(result)}")

✅ Grouped objectives.json written successfully.
📦 Total unique IDs: 35


In [16]:
from collections import defaultdict

# --- Load your existing objectives.json ---
with open("artifacts.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# --- Group by id ---
grouped = defaultdict(list)
for entry in data:
    grouped[str(entry["id"])].extend(entry["artifacts"])

# --- Convert to the correct format ---
result = [{"id": k, "artifacts": v} for k, v in grouped.items()]

# --- Save back to a new JSON file ---
with open("artifacts_grouped.json", "w", encoding="utf-8") as f:
    json.dump(result, f, indent=2, ensure_ascii=False)

print("✅ Grouped artifacts.json written successfully.")
print(f"📦 Total unique IDs: {len(result)}")

✅ Grouped artifacts.json written successfully.
📦 Total unique IDs: 35


In [17]:
import os
import shutil
from pathlib import Path

def move_json_files(destination_relative="../html_workspace", overwrite=True):
    """
    Move all .json files from the script directory (source) into destination_relative
    (which is interpreted relative to the script directory). Works in interactive
    interpreters too (falls back to current working directory).
    """

    # Resolve source folder robustly:
    if "__file__" in globals():
        # When script is run as a file
        source_folder = Path(__file__).resolve().parent
    else:
        # When running in interactive session / notebook
        source_folder = Path.cwd()

    # Resolve destination relative to the source folder
    destination_folder = (source_folder / destination_relative).resolve()

    # Ensure destination exists
    destination_folder.mkdir(parents=True, exist_ok=True)

    # Find JSON files in source folder
    json_files = [p for p in source_folder.iterdir() if p.is_file() and p.suffix.lower() == ".json"]

    if not json_files:
        print(f"⚠️ No JSON files found in {source_folder}")
        return

    print(f"📦 Found {len(json_files)} JSON file(s) in {source_folder} — moving to {destination_folder}")

    for src_path in json_files:
        dest_path = destination_folder / src_path.name

        if dest_path.exists() and not overwrite:
            base = src_path.stem
            ext = src_path.suffix
            counter = 1
            while (destination_folder / f"{base}_{counter}{ext}").exists():
                counter += 1
            dest_path = destination_folder / f"{base}_{counter}{ext}"

        shutil.move(str(src_path), str(dest_path))
        print(f"✅ Moved {src_path.name} → {dest_path}")

    print("🎉 Done.")

if __name__ == "__main__":
    # default destination_relative is "../folder_2" (one level up into folder_2)
    # you can change it, e.g. move_json_files(destination_relative="../../data/folder2")
    move_json_files()


📦 Found 7 JSON file(s) in /home/violet/Documents/projects/it_cmf/database_workspace — moving to /home/violet/Documents/projects/it_cmf/html_workspace
✅ Moved nodes.json → /home/violet/Documents/projects/it_cmf/html_workspace/nodes.json
✅ Moved objectives.json → /home/violet/Documents/projects/it_cmf/html_workspace/objectives.json
✅ Moved artifacts.json → /home/violet/Documents/projects/it_cmf/html_workspace/artifacts.json
✅ Moved artifacts_grouped.json → /home/violet/Documents/projects/it_cmf/html_workspace/artifacts_grouped.json
✅ Moved cbblinks.json → /home/violet/Documents/projects/it_cmf/html_workspace/cbblinks.json
✅ Moved objectives_grouped.json → /home/violet/Documents/projects/it_cmf/html_workspace/objectives_grouped.json
✅ Moved links.json → /home/violet/Documents/projects/it_cmf/html_workspace/links.json
🎉 Done.
